In [2]:
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, CrossEncoder, util,SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
import pandas as pd
import numpy as np
import torch
from datasets import Dataset

# Bước 1: Đọc dataset NLI và QA sau đó tách content ra làm file Corpus

In [4]:
import re

def clean_text(data):
    """
    Xử lý chuỗi để:
    - Giữ lại phần text trong [**text**] hoặc [text].
    - Bỏ phần (https://...).
    
    Args:
        data (str): Chuỗi cần xử lý.
    
    Returns:
        str: Chuỗi đã được làm sạch.
    """
    # Bước 1: Lấy phần text trong [**text**] hoặc [text]
    data = re.sub(r'\[\*\*(.*?)\*\*\]|\[(.*?)\]', lambda m: m.group(1) or m.group(2), data)
    
    # Bước 2: Loại bỏ phần (https://...)
    data = re.sub(r'\(https?://[^\)]+\)', '', data)
    
    # Loại bỏ các khoảng trắng thừa (nếu có)
    return data.strip()

# splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# df_NLI = pd.read_parquet("hf://datasets/BookingCare/ViHealth-NLI/" + splits["train"])
# splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet', 'val': 'data/val-00000-of-00001.parquet'}
# df_QA = pd.read_parquet("hf://datasets/BookingCare/ViHealthQA/" + splits["train"])
# df_QA_test = pd.read_parquet("hf://datasets/BookingCare/ViHealthQA/" + splits["test"])
# df_QA_val = pd.read_parquet("hf://datasets/BookingCare/ViHealthQA/" + splits["val"])

# # Áp dụng hàm xử lý cho cột `context`
# df_NLI["sentence2"] = df_NLI["sentence2"].apply(clean_text)
# df_QA["document"] = df_QA["document"].apply(clean_text)
# df_QA_test["document"] = df_QA_test["document"].apply(clean_text)
# df_QA_val["document"] = df_QA_val["document"].apply(clean_text)

# # 1. Lấy dữ liệu từ cột document và sentence2
# documents = df_QA['document']
# documents1 = df_QA_test['document']
# documents2 = df_QA_val['document']
# sentences = df_NLI['sentence2']

# # 2. Nối dữ liệu hai cột lại với nhau
# # Ghép cả hai cột thành một DataFrame mới
# corpus = pd.DataFrame({
#     'context': pd.concat([documents,documents1,documents2, sentences]).reset_index(drop=True)
# })

# # # 3. Lưu vào file CSV hoặc TXT
# corpus.to_csv('/kaggle/working/corpus.csv', index=False)  # Lưu thành file CSV
# # df_QA.to_csv('/kaggle/working/train.csv', index=False)
# # df_QA_test.to_csv('/kaggle/working/test.csv', index=False)
# # df_QA_val.to_csv('/kaggle/working/val.csv', index=False)

# # print("Dữ liệu đã được lưu vào file corpus.csv")

In [5]:
Corpus = pd.read_csv("/kaggle/input/healthcare-chatbot/corpus_healthcare.csv")
train = pd.read_csv("/kaggle/input/healthcare-chatbot/train_healthcare.csv")
test = pd.read_csv("/kaggle/input/healthcare-chatbot/test_healthcare.csv")
val = pd.read_csv("/kaggle/input/healthcare-chatbot/val_healthcare.csv")

***corpus dài nhất có: 4437 token***

In [6]:
bi_encoder = SentenceTransformer("/kaggle/input/halong_embedding_4jan/other/default/1/checkpoint-3500").cuda()

answers = list(Corpus["context"])

# Chỉ giữ các phần tử là chuỗi
answers = [str(answer) for answer in Corpus["context"] if isinstance(answer, (str, int, float))]

question ='''Vì sao phải gây mê khi chụp MRI cho trẻ?'''

# query_embedding = bi_encoder.encode(question, convert_to_tensor=True)
# doc_embeddings = bi_encoder.encode(answers, convert_to_tensor=True)

doc_embeddings = torch.load("/kaggle/input/halong_embedding/other/default/1/doc_embeddings_hl_finetune.pt")

# torch.save(doc_embeddings, '/kaggle/working/doc_embeddings_hl_ft.pt')


<ipython-input-6-6b0eabe49f61>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  doc_embeddings = torch.load("/kaggle/input/halong_embedding/other/default/1/doc_embeddings_h

# Bước 3: Cros-encoder

In [7]:
# Kiểm tra nếu GPU có sẵn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Bước 1: Tải mô hình Bi-Encoder
en = "/kaggle/input/bge_finetune_6file/transformers/default/1/bge_reranker_finetune_24Dec_latest5"  # "/kaggle/input/viranker/other/26_dec/1"

cross_encoder = AutoModelForSequenceClassification.from_pretrained(en)
cross_encoder = cross_encoder.to(device)   # Chuyển Cross-Encoder sang GPU

tokenizer = AutoTokenizer.from_pretrained(en)

# TÍNH MRR BIENCODER

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch

def calculate_mrr(relevant_indices):
    """
    Tính Mean Reciprocal Rank (MRR) cho bi-encoder.
    
    Parameters:
    - relevant_indices: Danh sách chỉ mục các câu trả lời đúng (list of int)
    
    Returns:
    - mrr: Giá trị Mean Reciprocal Rank (float)
    """
    # Encode queries và corpus
    query_embeddings = torch.load("/kaggle/input/query_embedding_hl/other/default/2/query_embeddings_hl_ft.pt")[:5000]
    corpus_embeddings = torch.load("/kaggle/input/halong_embedding/other/default/1/doc_embeddings_hl_finetune.pt")
    
    # Chuyển embeddings từ GPU sang CPU nếu cần
    query_embeddings = query_embeddings.cpu().numpy() if query_embeddings.is_cuda else query_embeddings.numpy()
    corpus_embeddings = corpus_embeddings.cpu().numpy() if corpus_embeddings.is_cuda else corpus_embeddings.numpy()
    
    # Tính similarity scores giữa query và corpus
    scores = cosine_similarity(query_embeddings, corpus_embeddings)
    
    # Tính Reciprocal Rank cho từng query
    reciprocal_ranks = []
    for i, relevant_idx in enumerate(relevant_indices):
        query_scores = scores[i]
        ranked_indices = np.argsort(query_scores)[::-1]
        rank_position = np.where(ranked_indices == relevant_idx)[0][0] + 1  # Vị trí bắt đầu từ 1
        reciprocal_ranks.append(1 / rank_position)
    
    # Tính Mean Reciprocal Rank
    mrr = np.mean(reciprocal_ranks)
    return mrr

# Ví dụ sử dụng
relevant_indices = list(range(0, 112371))
mrr = calculate_mrr(relevant_indices[:5000])
print(f"MRR: {mrr}")

# TÍNH MRR CROSSENCODER

In [ ]:
import torch
import torch.nn.functional as F
from sentence_transformers import util

# Giả sử bạn đã có các biến sau:
# - train: chứa dữ liệu với các cột "query" và "document".
# - bi_encoder: mô hình Bi-Encoder đã được huấn luyện.
# - cross_encoder: mô hình Cross-Encoder đã được huấn luyện.
# - doc_embeddings: embedding của toàn bộ tài liệu (answers).
# - answers: danh sách tài liệu (corpus).

# Số mẫu train để tính MRR
n_samples = 1000

def calculate_mrr(train, bi_encoder, cross_encoder, doc_embeddings, answers, n_samples):
    mrr = 0.0
    top_k_bi = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Duyệt qua từng mẫu trong tập train
    for idx in range(n_samples):
        question = train["query"][idx]
        correct_answer = train["document"][idx]

        # 1. Bi-Encoder: Lấy top-k tài liệu
        query_embedding = bi_encoder.encode(question, convert_to_tensor=True)
        bi_scores = util.cos_sim(query_embedding, doc_embeddings)[0].tolist()
        top_indices_bi = sorted(range(len(bi_scores)), key=lambda i: bi_scores[i], reverse=True)[:top_k_bi]
        top_docs_bi = [(answers[i], bi_scores[i]) for i in top_indices_bi]

        # 2. Cross-Encoder: Tái xếp hạng top-k
        cross_doc = [[question, doc] for doc, _ in top_docs_bi]

        with torch.no_grad():
            inputs = tokenizer(cross_doc, padding=True, truncation=True, return_tensors='pt', max_length=512)
            inputs = {key: value.to(device) for key, value in inputs.items()}
            outputs = cross_encoder(**inputs, return_dict=True)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze()

        # Gộp kết quả với điểm Cross-Encoder
        top_results = [
            (doc[0], doc[1], probabilities[i].item())
            for i, doc in enumerate(top_docs_bi)
        ]
        
        # Sắp xếp lại theo điểm Cross-Encoder
        top_results = sorted(top_results, key=lambda x: x[2], reverse=True)

        # 3. Tìm vị trí của tài liệu đúng
        rank = 0
        for i, (doc, _, _) in enumerate(top_results):
            if doc == correct_answer:
                rank = i + 1
                break

        # 4. Tính Reciprocal Rank
        if rank > 0:
            mrr += 1.0 / rank
        else:
            mrr += 1.0/10.0

    # 5. Tính Mean Reciprocal Rank (MRR)
    mrr /= n_samples
    return mrr

# Gọi hàm tính MRR
mrr_score = calculate_mrr(train, bi_encoder, cross_encoder, doc_embeddings, answers, n_samples=1000)

In [ ]:

print(f"MRR for the first {n_samples} samples: {mrr_score:.4f}")
